## STEP 0: LIBRARIES IMPORT

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## STEP 1: IMPORT DATASET

In [ ]:
movie_titles_df = pd.read_csv('Movie_ID_Titles')

In [ ]:
movie_titles_df

In [ ]:
movies_rating_df = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

In [ ]:
movies_rating_df

In [ ]:
movies_rating_df.drop(['timestamp'], axis = 1, inplace = True)

In [ ]:
movies_rating_df

In [ ]:
movies_rating_df.describe()

In [ ]:
movies_rating_df.info()

In [ ]:
movies_rating_df = pd.merge(movies_rating_df, movie_titles_df, on = 'item_id')

In [ ]:
movies_rating_df

## STEP 2: VISUALIZE THE DATASET

In [ ]:
movies_rating_df.groupby('title').describe()

In [ ]:
movies_rating_df.groupby('title')['rating'].describe()

In [ ]:
ratings_df_mean = movies_rating_df.groupby('title')['rating'].describe()['mean']

In [ ]:
ratings_df_mean

In [ ]:
ratings_df_count = movies_rating_df.groupby('title')['rating'].describe()['count']

In [ ]:
ratings_df_count

In [ ]:
ratings_mean_count_df = pd.concat([ratings_df_count, ratings_df_mean], axis = 1)

In [ ]:
ratings_mean_count_df

In [ ]:
ratings_mean_count_df.reset_index()

In [ ]:
ratings_mean_count_df['mean'].plot(bins = 100, kind = 'hist', color = 'r')

In [ ]:
ratings_mean_count_df['count'].plot(bins = 100, kind = 'hist', color = 'r')

In [ ]:
ratings_mean_count_df[ratings_mean_count_df['mean'] == 5]

In [ ]:
ratings_mean_count_df.sort_values('count', ascending = False).head(100)

## STEP 3: PERFORM ITEM-BASED COLLABORATIVE FILTERING ON ONE MOVIE SAMPLE

In [ ]:
userid_movietitle_matrix = movies_rating_df.pivot_table(index = 'user_id', columns = 'title', values = 'rating') 

In [ ]:
userid_movietitle_matrix

In [ ]:
titanic = userid_movietitle_matrix['Titanic (1997)']

In [ ]:
titanic

In [ ]:
starwars = userid_movietitle_matrix['Star Wars (1977)']

In [ ]:
starwars

In [ ]:
# Let's calculate the correlations
titanic_correlations = pd.DataFrame(userid_movietitle_matrix.corrwith(titanic), columns=['Correlation'])
titanic_correlations = titanic_correlations.join(ratings_mean_count_df['count'])


In [ ]:
titanic_correlations

In [ ]:
titanic_correlations.dropna(inplace = True)
titanic_correlations

In [ ]:
titanic_correlations.sort_values('Correlation', ascending = False)

In [ ]:
titanic_correlations[titanic_correlations['count']>80].sort_values('Correlation', ascending = False).head()

## STEP 4: CREATE AN ITEM-BASED COLLABORATIVE FILTER ON THE ENTIRE DATASET

In [ ]:
userid_movietitle_matrix

In [ ]:
movie_correlations = userid_movietitle_matrix.corr(method = 'pearson', min_periods = 80)

In [ ]:
movie_correlations

In [ ]:
myRatings = pd.read_csv('My_Ratings.csv')

In [ ]:
myRatings

In [ ]:
myRatings['Movie Name'][0]

In [ ]:
similiar_movies_list = pd.Series()

for i in range(0,2):
    similiar_movie = movie_correlations[  myRatings['Movie Name'][i]  ].dropna()
    similiar_movie = similiar_movie.map(lambda x: x* myRatings ['Ratings'][i])
    similiar_movies_list = similiar_movies_list.append(similiar_movie)
    

In [ ]:
similiar_movies_list.sort_values(inplace = True, ascending = False)
print(similiar_movies_list.head(10))